In [1]:
import os
from datetime import datetime

import pandas as pd
from feast import FeatureStore

In [2]:
raw_data_path = os.path.join("feature_store", "feature_repo", "data", "driver_stats.parquet")
feature_store_path = os.path.join("feature_store", "feature_repo")

### Check data

In [3]:
df = pd.read_parquet(raw_data_path)

In [4]:
df.head(5)

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-10-17 12:07:08.228578+00:00,1001,1.000000,1.000000,1000,2024-10-17 12:07:08.228581
1,2024-10-02 11:00:00+00:00,1005,0.429879,0.194598,582,2024-10-17 11:30:07.072000
2,2024-10-02 12:00:00+00:00,1005,0.230119,0.642878,551,2024-10-17 11:30:07.072000
3,2024-10-02 13:00:00+00:00,1005,0.128600,0.674187,38,2024-10-17 11:30:07.072000
4,2024-10-02 14:00:00+00:00,1005,0.400603,0.473636,583,2024-10-17 11:30:07.072000


### Features inference

In [5]:
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
    }
)

In [6]:
entity_df

,driver_id,event_timestamp,label_driver_reported_satisfaction
0,1001,2021-04-12 10:59:42,1
1,1002,2021-04-12 08:12:10,5
2,1003,2021-04-12 16:40:26,3


In [7]:
store = FeatureStore(repo_path=feature_store_path)

In [ ]:
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
).to_df()

print("----- Feature schema -----\n")
print(training_df.info())

In [ ]:
training_df.head()

### Features View - on demand

In [ ]:
entity_df = pd.DataFrame.from_dict(
    {
        # entity's join key -> entity values
        "driver_id": [1001, 1002, 1003],
        # "event_timestamp" (reserved key) -> timestamps
        "event_timestamp": [
            datetime(2021, 4, 12, 10, 59, 42),
            datetime(2021, 4, 12, 8, 12, 10),
            datetime(2021, 4, 12, 16, 40, 26),
        ],
        # (optional) label name -> label values. Feast does not process these
        "label_driver_reported_satisfaction": [1, 5, 3],
        # values we're using for an on-demand transformation
        "val_to_add": [1, 2, 3],
        "val_to_add_2": [10, 20, 30],
    }
)

In [ ]:
entity_df.head(5)

In [ ]:
training_df = store.get_historical_features(
    entity_df=entity_df,
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
        "transformed_conv_rate:conv_rate_plus_val1",
        "transformed_conv_rate:conv_rate_plus_val2",
    ],
).to_df()

In [ ]:
training_df

In [ ]:
# Online feature retrieval
online_features = store.get_online_features(
    features=[
        "driver_hourly_stats:conv_rate",
        "driver_hourly_stats:acc_rate",
        "driver_hourly_stats:avg_daily_trips",
    ],
    entity_rows=[{"driver_id": 1001}, {"driver_id": 1002}],
).to_dict()

print("Online features for drivers 1001, 1002:")
for key, value in online_features.items():
    print(f"{key}: {value}")

In [ ]:
# Using Feature Service for consistent feature sets
training_df_v1 = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("driver_activity_v1")
).to_df()

print("\nFeatures from driver_activity_v1 service:")
training_df_v1.head()

In [ ]:
# Get feature view metadata
feature_view = store.get_feature_view("driver_hourly_stats")
print("\nFeature view metadata:")
print(f"Name: {feature_view.name}")
print(f"Entities: {feature_view.entities}")
print(f"TTL: {feature_view.ttl}")
print(f"Online: {feature_view.online}")
print(f"Features: {[f.name for f in feature_view.features]}")

In [11]:
# Using Feature Service for consistent feature sets
training_df_v4 = store.get_historical_features(
    entity_df=entity_df,
    features=store.get_feature_service("driver_activity_v4")
).to_df()

print("\nFeatures from driver_activity_v4 service:")
training_df_v4.head()


Features from driver_activity_v4 service:


,driver_id,event_timestamp,label_driver_reported_satisfaction,conv_rate,acc_rate,avg_daily_trips,combined_rating,performance_score
0,1001,2021-04-12 10:59:42+00:00,1,0.709758,0.692957,402,0.703038,4.217479
1,1002,2021-04-12 08:12:10+00:00,5,0.718295,0.584081,370,0.664610,3.931966
2,1003,2021-04-12 16:40:26+00:00,3,0.697411,0.197680,25,0.497519,1.620965
